In [2]:
from tensorflow import keras
from tensorflow.keras import layers
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import image_dataset_from_directory
import matplotlib.pyplot as plt
# from tensorflow.keras.models import load_model
from keras.preprocessing import image 
import numpy as np
import os
import pandas as pd

In [6]:
## to download datset directly from kaggle:

# !kaggle competitions download -c dogs-vs-cats
# import zipfile
# zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip', 'r')
# zip_ref.extractall('/content')
# zip_ref.close()

403 - Forbidden - Permission 'datasets.get' was denied
